In [1]:
!pwd

/lustre/home/opacheco/MEDA_Challenge/notebooks


In [2]:
# Primero desinstala la versión de GPU si la tienes
!pip uninstall -y onnxruntime-gpu
# Instala la versión de CPU
!pip install onnxruntime

Found existing installation: onnxruntime-gpu 1.19.2
Uninstalling onnxruntime-gpu-1.19.2:
  Successfully uninstalled onnxruntime-gpu-1.19.2
ERROR: Exception:
Traceback (most recent call last):
  File "/lustre/proyectos/p032/env/lib64/python3.9/site-packages/pip/_internal/cli/base_command.py", line 173, in _main
    status = self.run(options, args)
  File "/lustre/proyectos/p032/env/lib64/python3.9/site-packages/pip/_internal/commands/uninstall.py", line 97, in run
    uninstall_pathset.commit()
  File "/lustre/proyectos/p032/env/lib64/python3.9/site-packages/pip/_internal/req/req_uninstall.py", line 436, in commit
    self._moved_paths.commit()
  File "/lustre/proyectos/p032/env/lib64/python3.9/site-packages/pip/_internal/req/req_uninstall.py", line 287, in commit
    save_dir.cleanup()
  File "/lustre/proyectos/p032/env/lib64/python3.9/site-packages/pip/_internal/utils/temp_dir.py", line 184, in cleanup
    rmtree(self._path)
  File "/lustre/proyectos/p032/env/lib64/python3.9/site-pack

In [3]:
## Importando librerias
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, Subset
import numpy as np
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import copy
import onnxruntime as ort

In [4]:
## Configuracion de Variables
# ========================================================
# NUEVO: Ruta a tu modelo ONNX
# Este modelo actuará como el backbone extractor de características.
PATH_ONNX_MODEL = "../models/modelo_chicas.onnx"
# NUEVO: Define el número de características que produce tu modelo ONNX.
# Por ejemplo, para un ResNet-18, esto sería 512.
ONNX_OUTPUT_FEATURES = 6
# ========================================================


# Apunta al directorio raíz del dataset
PATH_DATASET = "/lustre/home/opacheco/MEDA_Challenge/compartido/datasets/daudon_dataset/test"

# Define el número de clases de tu dataset
NUM_CLASES = 6

# Parámetros para el entrenamiento del cabezal
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 1.0 - TRAIN_RATIO - VAL_RATIO
K_SHOTS = 50
EPOCHS = 50 # Ajusta según sea necesario
BATCH_SIZE = 1024 # Ajusta según sea necesario

In [5]:
# --------------------------------------------------------
# 1. FIJAR SEMILLA ALEATORIA
# --------------------------------------------------------
SEED = 46
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
# ========================================================
# NUEVO: Clase Wrapper para el Backbone ONNX
# Esta clase carga el modelo ONNX y lo hace compatible con PyTorch.
# ========================================================
class OnnxBackbone(nn.Module):
    def __init__(self, onnx_path):
        super().__init__()
        print(f"Cargando modelo ONNX desde: {onnx_path}")
        # Cargar el modelo en una sesión de ONNX Runtime, priorizando GPU
        self.session = ort.InferenceSession(onnx_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
        
        # Obtener los nombres de entrada y salida del modelo
        self.input_name = self.session.get_inputs()[0].name
        self.output_name = self.session.get_outputs()[0].name
        print(f"✅ Modelo ONNX cargado. Input: '{self.input_name}', Output: '{self.output_name}'")

    def forward(self, x):
        # 1. Convertir el tensor de PyTorch (posiblemente en GPU) a un array de NumPy en CPU
        x_np = x.cpu().numpy()

        # 2. Ejecutar la inferencia con ONNX Runtime
        # La salida es una lista de arrays de NumPy
        output_np = self.session.run([self.output_name], {self.input_name: x_np})[0]

        # 3. Convertir el resultado de NumPy de vuelta a un tensor de PyTorch
        # y moverlo al dispositivo original (CPU o GPU)
        output_tensor = torch.from_numpy(output_np).to(x.device)
        
        return output_tensor

In [7]:
# --------------------------------------------------------
# 2. DATASET Y DATALOADERS (Sin cambios)
# --------------------------------------------------------
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

try:
    full_dataset = ImageFolder(PATH_DATASET, transform=data_transform)
    print(f"Dataset cargado: {len(full_dataset)} imágenes en {len(full_dataset.classes)} clases.")
    NUM_CLASES = len(full_dataset.classes)
except FileNotFoundError:
    print(f"Error: No se encontró el directorio {PATH_DATASET}")
    raise

# División estratificada del dataset
targets = full_dataset.targets
indices = list(range(len(targets)))
train_indices, val_test_indices, train_targets, val_test_targets = train_test_split(
    indices, targets, train_size=TRAIN_RATIO, stratify=targets, random_state=SEED)
test_split_ratio = TEST_RATIO / (VAL_RATIO + TEST_RATIO)
val_indices, test_indices = train_test_split(
    val_test_indices, test_size=test_split_ratio, stratify=val_test_targets, random_state=SEED)

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)
test_dataset = Subset(full_dataset, test_indices)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Creación del dataset Few-Shot
print(f"Creando Support Set de {K_SHOTS}-shot...")
support_indices = []
for c in range(NUM_CLASES):
    class_indices = [i for i, t in zip(train_indices, train_targets) if t == c]
    if len(class_indices) < K_SHOTS:
        k_shot_indices = class_indices
    else:
        k_shot_indices = random.sample(class_indices, K_SHOTS)
    support_indices.extend(k_shot_indices)

few_shot_dataset = Subset(full_dataset, support_indices)
few_shot_train_loader = DataLoader(few_shot_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
print(f"DataLoader Few-Shot creado con {len(few_shot_dataset)} imágenes.")

Dataset cargado: 2400 imágenes en 6 clases.
Creando Support Set de 50-shot...
DataLoader Few-Shot creado con 300 imágenes.


/lustre/proyectos/p032/env/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
# --------------------------------------------------------
# 3. CABEZAL DE CLASIFICACIÓN (Adaptado para recibir cualquier backbone)
# --------------------------------------------------------
class ClassificationModel(nn.Module):
    def __init__(self, backbone, num_features, num_classes):
        super().__init__()
        # El backbone será nuestro modelo ONNX envuelto en la clase OnnxBackbone
        self.backbone = backbone

        # Cabezal de clasificación (solo estas capas serán entrenadas)
        self.linear_head = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.LayerNorm(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # Extraer características con el backbone (SIN calcular gradientes para él)
        with torch.no_grad():
            feats = self.backbone(x)
        
        # Aplanar las características para el cabezal lineal
        feats = feats.view(feats.size(0), -1)
        
        # Pasar las características por el cabezal que sí se entrena
        out = self.linear_head(feats)
        return out
    
# === Configuración del modelo con ONNX ===
# 1. Instanciar el backbone ONNX
onnx_backbone = OnnxBackbone(PATH_ONNX_MODEL)

# 2. Crear el modelo de clasificación completo
model = ClassificationModel(
    backbone=onnx_backbone,
    num_features=ONNX_OUTPUT_FEATURES,
    num_classes=NUM_CLASES
).to(DEVICE)

# 3. Configurar pérdida y optimizador
# El optimizador solo actualizará los pesos del `linear_head`
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.linear_head.parameters(), lr=3e-3, weight_decay=1e-4)

Cargando modelo ONNX desde: ../models/modelo_chicas.onnx


/lustre/proyectos/p032/env/lib64/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-11-05 05:38:37.342786874 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 879614, index: 6, mask: {19, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-11-05 05:38:37.346378443 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 879615, index: 7, mask: {20, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-11-05 05:38:37.351367564 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 879616, index: 8, mask: {21, }, error code: 22 error msg: Inval

✅ Modelo ONNX cargado. Input: 'images', Output: 'logits6'


In [ ]:
# --------------------------------------------------------
# 4. ENTRENAMIENTO (Sin cambios en la lógica)
# --------------------------------------------------------
print("\n🚀 Iniciando entrenamiento del cabezal de clasificación con backbone ONNX...")

best_val_acc = 0.0
best_weights = None
patience = 30
wait = 0

for epoch in range(EPOCHS):
    model.train() # Pone en modo de entrenamiento solo el cabezal
    running_loss = 0.0

    for inputs, labels in few_shot_train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(few_shot_train_loader.dataset)

    # Validación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total

    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {epoch_loss:.4f} - Val Acc: {val_acc:.2f}%")

    # Early stopping y guardado del mejor modelo
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_weights = copy.deepcopy(model.state_dict())
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print(f"⏹ Early stopping en epoch {epoch+1}")
            break

print("Entrenamiento finalizado.")
if best_weights:
    model.load_state_dict(best_weights)
    print(f"✅ Mejor modelo cargado con Val Acc: {best_val_acc:.2f}%")


🚀 Iniciando entrenamiento del cabezal de clasificación con backbone ONNX...


In [ ]:
# --------------------------------------------------------
# 5. EVALUACIÓN FINAL Y MATRIZ DE CONFUSIÓN
# --------------------------------------------------------
print("\n🧪 Evaluando en el set de testeo...")
model.eval()
all_labels, all_preds = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

accuracy = 100 * (np.array(all_preds) == np.array(all_labels)).sum() / len(all_labels)
print("\n========================================================")
print(f"🎉 ¡Evaluación con Backbone ONNX completa! 🎉")
print(f"   Accuracy en test: {accuracy:.2f} %")
print("========================================================")

# Matriz de Confusión
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=full_dataset.classes, yticklabels=full_dataset.classes)
plt.title('Matriz de Confusión (Backbone ONNX)')
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.show()